Import necessary packages

In [1]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
import json
import nest_asyncio
nest_asyncio.apply()


#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "Context_files"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)

#create inputs to the model, telling it what needs to be done. 
#provides system message
system1 = context_gen("system1.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

system3 = context_gen("system3.txt")

system2 = context_gen("system2.txt")

rag_prompt = context_gen("rag_prompt.txt")


#First input to the model, breaks query down and provides geometry definition

class analyzed_query(BaseModel):
    """Identifying whether a geometric feature is present in the database and stating the name of the feature"""
    answer: str= Field(description="YES or NO")
    features: List= Field(description="Name of the identified geometrical feature")

class Response(BaseModel):
    """Listing the answer of whether the feature is present in the database and the name of the feature"""
    Response: List[analyzed_query]
    
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [8]:
'''change the human variable to test different inputs'''
human = ("select all blend faces and then select their edges")

parser = PydanticOutputParser(pydantic_object=analyzed_query)
prompt = ChatPromptTemplate.from_messages([("system", rag_prompt + "Answer the user query. Print the output in Json format according to the instructions.\n{format_instructions}",), ("human", human),]).partial(format_instructions=parser.get_format_instructions())
chain = prompt | chat | parser
feature_extractor = chain.invoke({})
answer = feature_extractor.answer

extracted_features = feature_extractor.features
print(answer, "\n", extracted_features)

YES 
 ['blend faces', 'edges']


In [9]:
loader = DirectoryLoader("RAG")
raw_docs = loader.load()
textsplitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = textsplitter.split_documents(raw_docs)
db = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = db.as_retriever()
 

prompt_for_rag = ChatPromptTemplate.from_template("""You are an assistant that is an expert at RAG (Retrieval Augmented Generation). You have been tasked with looking for the definition of the word provided to you as input and generate a summary of the available information."
            You will use only the context provided to you.                                 
            Context:{context}
            Query_to_db:{query}""")
rag_chain = ({"context": retriever, "query": RunnablePassthrough()}) | prompt_for_rag | chat | StrOutputParser()

In [10]:
#Base prompt chain, generates the first yaml version with minimal context

base_prompt = ChatPromptTemplate.from_messages([("system", system1), few_shot_prompt, MessagesPlaceholder(variable_name="messages")])
yaml_generator = base_prompt | chat 
rag_output = rag_chain.invoke(f"What is the definition of {extracted_features}?")
print(rag_output)

def chain_yes():
    chain = yaml_generator | StrOutputParser()
    yaml = chain.invoke({"messages": [human]}, {"tags": ["chain_yes"]})
    print("RAG not required, generating YAML")
    print()
    print(yaml)
    print()
    return yaml

def chain_no():
    chain = yaml_generator | StrOutputParser()
    ai = AIMessage(content=rag_output)
    yaml = chain.invoke({"messages":[ai]}, {"tags": ["chain_no"]})
    print("Getting more information using RAG, then generating YAML")
    print()
    print(yaml)
    print()
    return yaml

def decision_maker(response_obj: analyzed_query):
    for item in response_obj:
        if item == "YES":
            return chain_yes()
        elif item == "NO":
            return chain_no()


The definition of "blend faces" and "edges" is not explicitly provided in the given context. However, based on the information available, we can infer that "edges" refer to the linear features that connect various components in a CAD model, and "blend faces" could potentially refer to the process of smoothly transitioning or merging planar surfaces together.


In [5]:
model_output = []
for answer in feature_extractor:
    model_output.append(decision_maker(answer))

RAG not required, generating YAML

- filter: select * from faces where type like "Blend"
- expand: faceedges



- filter: select * from faces where type like "Blend"
- expand: faceedges
---

The provided YAML code is accurate and fulfills the user's request to select all blend faces and then select their edges. The code effectively filters the faces based on the type "Blend" and then expands to select the corresponding edges. 

Well done! The code is clear and directly addresses the user's query.
---

- filter: select * from faces where type like "Blend"
- expand: faceedges
---

The YAML code provided accurately selects all blend faces and then proceeds to select their edges. The filtering condition for the type "Blend" is correctly applied, followed by the expansion to include the corresponding edges.

The code is concise and effectively addresses the user's request. Great job!
---

